Muslimov Arthur, Last Checkpoint: 03/31/2020

Некоторые наиболее интересные исследования выполняются     <br/>
благодарся объеденению нескольких источников данных.       <br/>
И это может быть не просто приклеивание одного объекта     <br/>
к другиму. Эти операции включают в себя и более сложные    <br/>
соединения и слияния в стиле баз данных, корректно         <br/>
обрабатывающих все возможные ситуации совпадения наборов.  <br/>
Но и тут за нас сделали большую часть грязной работы.      <br/>
Объекты Series и DataFrame сделаны с расчётом на такие     <br/>
операции, и нам остаётся лишь дёргать за нужные ниточки,   <br/>
пока Pandas, подобно буксиру, тянет за собой сухогруз.

Как и пологается, сначала мы рассмотрим простую конкатенацию  <br/>
Series и DataFrame с помощью функции `pd.concat()`, и уже     <br/>
затем углубимся в недры Pandas за более запутанными           <br/>
реализациями соединений, выполняющихся в оперативной памяти.

Начнём с обыных импортов:

In [1]:
import numpy as np
import pandas as pd

%xmode Minimal
%autosave 0

rng = np.random.RandomState(42)  # для одинаковых случайностей

Exception reporting mode: Minimal


Autosave disabled


Для удобства напишем метод для быстрого создания DataFrame.

In [2]:
def make_df(cols, ind):
    """Быстро создаём объект DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}  # лучший способ для понимания подобных алгоритом -
                            # - простое выполнение его карандашом на листочке.
    return pd.DataFrame(data, ind)

# Ну и, конечно же, его испытание
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


# Напоминание: конкатенация массивов NumPy

Конкатенация Series и DataFrame очень похожа на конкатенацию     <br/>
в массивах NumPy. Делалась она там функцией `np.concatenate()`.

In [3]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])  # освежаем память

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

Первым на вход она принимает список массивов для объединения.  <br/>
Нам также доступен выбор оси, по которому будет склейка.

In [4]:
grid = [[1, 2],
        [3, 4]]
np.concatenate([grid, grid], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

# Простая конкатенация с помощью метода `pd.concat()`

Синтаксис `pd.concat()` аналогичен `np.concatenate()`, но при этом  <br/>
версия Pandas содержит гораздо больше параметров. О них позже.      

Если у тебя открыт этот Jupyter блокнот, то ты можешь поставить     <br/>
курсор на функцию и нажать Shift + Tab, чтобы увидеть её мануал.

In [5]:
ser1 = pd.Series(list('ABC'), index=[1, 2, 3])
ser2 = pd.Series(list('DEF'), index=[4, 5, 6])
pd.concat([ser1, ser2])  # также, как и np.concatenate() с массивами

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

Объекты с более большим числом измерений тоже хорошо сцепляются.

In [6]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
print("df1:\n", df1, end="\n\n")
print("df2:\n", df2, end="\n\n")
print("concat:\n", pd.concat([df1, df2]))

df1:
     A   B
1  A1  B1
2  A2  B2

df2:
     A   B
3  A3  B3
4  A4  B4

concat:
     A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


Как ты видишь, также, как и `np.concatenate()`, `pd.concat()`  <br/>
по умолчанию склеивает построчно, т.е. по высшему измерению.   <br/>
Но тут также доступен параметр `axis`. Он даже чуть лучше.

In [7]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
print("df3:\n", df3, end="\n\n")
print("df4:\n", df4, end="\n\n")
print("concat:\n", pd.concat([df3, df4], axis='columns'))  # в Pandas параметр axis чуть мягче

df3:
     A   B
0  A0  B0
1  A1  B1

df4:
     C   D
0  C0  D0
1  C1  D1

concat:
     A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1


Ты, конечно, можешь, задать `axis=1`, но `'columns'` чуть более интуитивен.

### Дублирование индексов

Но в Pandas есть одна важная особенность - он очень не любит,      <br/>
когда теряются данные. Поэтому даже в случае, когда склеиваются    <br/>
два объекта с одинаковыми индексами, он не станет их выравнивать,  <br/>
рискуя потерять данные. Pandas ***сохраняет индексы***.

In [8]:
x = make_df('AB', [0, 1])
y = make_df('BC', [0, 1])
xy = pd.concat([x, y], axis='columns')  # просто припишет его рядом, без всякой оптимизации
xy

,A,B,B,C
0,A0,B0,B0,C0
1,A1,B1,B1,C1


Это допустимо, но нежелательно. При попытке получить данные  <br/>
с одинаковыми индексами, наш DataFrame выдаст оба значение.

In [9]:
xy['B']

,B,B
0,B0,B0
1,B1,B1


Фукнция `pd.concat()` предоставляет несколько выходов из данной ситуации.

**Перехват повторов как ошибка.** Если ты хочешь, чтобы при таком  <br/>
перекрытии сразу выскакивала ошибка, то в `pd.concat()` можешь     <br/>
включить `verify_integrity`, задав ему аргумент `True`.

In [10]:
try:
    pd.concat([x, y], axis='columns', verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Index(['B'], dtype='object')


**Игнорирование индекса.** Иногда индексы могут сами по себе  <br/>
ничего не значить, и их потеря не приведёт к неприятностям.   <br/>
В таких случаях можно попросту включить флаг `ignore_index`,  <br/>
заменяющий проблемные индексы на стандартную нумерацию.

In [11]:
pd.concat([x, y], axis='columns', ignore_index=True)
                                        # заменяются имена столбцов, т.к. там проблема.

,0,1,2,3
0,A0,B0,B0,C0
1,A1,B1,B1,C1


**Добавление ключей мультииндекса.** Ещё один вариант -  <br/>
параметр `keys` для задания меток источнков данных.

In [12]:
pd.concat([x, y], axis='columns', keys=['x', 'y'])  # добавляется верхний уровень,
                                                    # обозначающий источник

x       y    
    A   B   B   C
0  A0  B0  B0  C0
1  A1  B1  B1  C1

Теперь мы имеем мультииндексированный DataFrame, а это значит,  <br/>
что мы можем преобразовать его в нужное нам представление.

### Конкатенация с использованием соединений

Пока всё вроде-бы хорошо, но в реальных условиях данные  <br/>
из различных источников могут иметь различные наборы     <br/>
имён столбцов. На этот случай у `pd.concat()` также      <br/>
припасёны опции. Рассмотрим следующие DataFrame'мы.

In [13]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
print("df5:\n", df5, end="\n\n")
print("df6:\n", df6, end="\n\n")
print("concat:\n", pd.concat([df5, df6]))  # какие-то столбцы совпадают, какие-то нет

df5:
     A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6:
     B   C   D
3  B3  C3  D3
4  B4  C4  D4

concat:
      A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


По умолчанию элементы, о которых нет данных, помечаются   <br/>
NA-значениями. Чтобы изменить это поведение, можно        <br/>
указать одну из нескольких опций для параметров `join`    <br/>
и `join_axes` нашей функции. По умолчанию `join='outer'`  <br/>
и берутся абсолютно все данные, но есть возможность       <br/>
их отсеить, оставляя лишь полностью заполненные строки    <br/>
или столбцы. Для этого нужно задать `join='inner'`.

In [14]:
pd.concat([df5, df6], join='inner')  # это как встроенный dropna()

,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


Ещё одна опция предназначена для задания явным образом индексов  <br/>
оставшихся столбцов. Нужно лишь отправить их список в  `join_axes`.

In [15]:
try:
    pd.concat([df5, df6], join_axes=['A', 'B', 'C'])
except TypeError as e:
    print("TypeError:", e)

TypeError: concat() got an unexpected keyword argument 'join_axes'


Поправка: её выпелили. Вот, что должно было получиться из этого:

 <i></i> | A   | B  | C
:--|-----|:---|:--
1  | A1  | B1 | C1
2  | A2  | B2 | C2
3  | NaN | B3 | C3
4  | NaN | B4 | C4

Ну... зато это упращение. Наверное её мало кто использовал, или     <br/>
был лёгкий обходной путь. Её же не могли убрать просто так, верно?

### Метод append()

Непосредственная конкатенация массивов так распространена,   <br/>
что в объекты Series и DataFrame добавили метод `append()`,  <br/>
позволяющий сделать то же самое, но "без лишних слов".

In [16]:
df1.append(df2)  # меньше нажатий по клавишам, чем если набирать pd.concat([df1, df2])

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


Не забывай, что в отличии от методов `append()` и `extend()`   <br/>
списков Python, метод Pandas не изменяет исходный объект.      <br/>
Вместо этого он создаёт новый объект с объединёнными данными.  <br/>
Это значит, что он не эффективeн, если тебе необходимо         <br/>
склеить данные несколько раз. Точнее, он, видимо, таким был    <br/>
когда-то, ведь теперь он умеет объеденять данные сразу         <br/>
списком и не хуже, чем `pd.concat()`.

In [17]:
df1.append([df2, df2])  # если бы он выделял по буферу для каждого нового объекта,
                        # было бы не так эффективно. Хотя этого всё-ещё можно
                        # достичь так - df1.append(df2).append(df2)

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4
3,A3,B3
4,A4,B4


В следующем разделе мы рассмотрим другой подход к объединению  <br/>
данных из нескольких источников. Там будет явно поинтереснее,  <br/>
ведь ты узнаешь как выполняется слияние в стиле быз данных     <br/>
с помощью функции `pd.merge()`. Если хочешь побольше узнать    <br/>
про методы `concat()` и `append()`, то можешь полистать        <br/>
документацию на сайте Pandas. Там осталось много неизведанного!